In [16]:
## create a folder in /dcs/large/uxxxxxxx/videos/ called video_split. Inside this create three subdirectories: validate, train, test
#NOTE: if you want to run this more than once make sure you empty these directories!!

import cv2
import numpy as np
#from google.colab.patches import cv2_imshow
import re
import random

import shutil

MODEL_LOCATION = "/dcs/large/u1901447/models/"
VIDEO_SPLIT_LOCATION = "/dcs/large/u1901447/videos/video_split/"

#SET DEBUG FLAG TO TRUE TO NOT ALTER ANY FILES BUT INSTEAD OUTPUT WHAT SHOULD HAPPEN INSTEAD
debug = False

In [17]:
#split into test/train?
videoLocation = "/dcs/large/u1901447/videos/220611galleivnor_2_movie-001.mov"
labelLocation = "/dcs/large/u1901447/labels/galleinvorLabels.lbl"
with open(labelLocation) as f:
    lines = f.readlines()

label_tuples = list()
for line in lines:
  #print(line)
  label_match = re.search(r'(?<=Label.)(.*)(?=:)',line)
  #print(label_match.group())
  frame_match = re.search(r'(?<=:)(.*)(?=\n)',line)
  if frame_match is None:
    frame_match = re.search(r'(?<=:)(.*)',line)
  #print(frame_match.group())
  label_tuples.append((label_match.group(),int(frame_match.group())))

# print(label_tuples)

In [18]:
# Define the location of the video we are processing
# Define the folder we want to save the extracted fragments into
save_path = '/dcs/large/u1901447/videos/videoClips/'
# Setup a dict to store the counts of files of each label so we can have a unique file name for each typoe of label
label_indexer = dict()
all_labels = set(l[0] for l in label_tuples)
label_indexer = {l:0 for l in all_labels}

## label_tuples: tuple of all labels from label file ## 
## all_labels stores each of the unique labels in a set data type ##
## label_indexer: dictionary where key is the label val is the counter ##

In [19]:
video_names = []

updated_label = []

for label, frames_count in label_tuples:
  # print(label)
  # print(frames_count)
  # we need at least 15 frames to make a decent clip
  
  if frames_count > 14:
    # videocap_name = f"{save_path}{label}_{label_indexer[label]}.mp4"
    fname = f"{label}_{label_indexer[label]}"
    label_indexer[label]+=1
    video_names.append(fname)
    updated_label.append((label, frames_count))

updated_label_tuples = tuple(updated_label)
  
  

In [20]:

#70% train, 15% test, 15% validation
TRAIN_PERCENT = 70.0
TEST_PERCENT = 15.0
VALID_PERCENT = 15.0
# summation = TRAIN_PERCENT + TEST_PERCENT + VALID_PERCENT

assert (TRAIN_PERCENT + TEST_PERCENT + VALID_PERCENT) == 100.0

label_tuples = updated_label_tuples


for class_type in all_labels:
    #idxs_of_classtype stores the index(es) at which this class type occurs in label_tuples
    idxs_of_classtype = [idx for idx in range(len(label_tuples)) if class_type == label_tuples[idx][0]]
    
    #total_videos: stores the total number of videos of type class_type
    total_videos = len(idxs_of_classtype)

    train_amount = int(len(idxs_of_classtype) * TRAIN_PERCENT/100) #number of videos to use for training data of this class type
    test_amount = int(len(idxs_of_classtype) * TEST_PERCENT/100)  #number of videos to use for testing data of this class type
    valid_amount = int(len(idxs_of_classtype) * VALID_PERCENT/100) #number of videos to use for validation data of this class type
    while train_amount + test_amount + valid_amount < total_videos:
        train_amount += 1
        
    #could use random.sample?
    # random.sample(range(1, total_videos, )

    # shuffle and then let first x be train, next y be test, ...
    random.shuffle(idxs_of_classtype)
    train_idxs = []
    test_idxs = []
    valid_idxs = []
    
    assert train_amount + test_amount + valid_amount == total_videos

    video_amounts = [train_amount, test_amount, valid_amount]
    video_types = ["train", "test", "validate"]
    type_ptr = 0
    counter = 0
    max_counter = train_amount
    for i in range(total_videos):
        if i == max_counter:
            type_ptr += 1
            max_counter += video_amounts[type_ptr]
        if video_types[type_ptr] == "train":
            train_idxs.append(idxs_of_classtype[i])
        elif  video_types[type_ptr] == "test":
            test_idxs.append(idxs_of_classtype[i])
        else:
            valid_idxs.append(idxs_of_classtype[i])

    ## TODO: this is a lot of repeated code. xander pls don't kill me, thx
    for i in test_idxs:
        original = f"{save_path}{video_names[i]}.mp4"
        dir = VIDEO_SPLIT_LOCATION + "test/"
        target = f"{dir}{video_names[i]}.mp4"

        if debug:
            print("copy", original, "to", target)
        else:
            shutil.copyfile(original, target)

    for i in train_idxs:
        original = f"{save_path}{video_names[i]}.mp4"
        dir = VIDEO_SPLIT_LOCATION + "train/"
        target = f"{dir}{video_names[i]}.mp4"

        if debug:
            print("copy", original, "to", target)
        else:
            shutil.copyfile(original, target)

    for i in valid_idxs:
        # assert class_type in video_names[i] 
        original = f"{save_path}{video_names[i]}.mp4"
        dir = VIDEO_SPLIT_LOCATION + "validate/"
        target = f"{dir}{video_names[i]}.mp4"

        if debug:
            print("copy", original, "to", target)
        else:
            shutil.copyfile(original, target)

1259
1066
1066
PASS_R
[10, 16, 21, 27, 44, 47, 60, 61, 65, 66, 106, 123, 167, 181, 194, 195, 211, 218, 227, 229, 233, 236, 240, 244, 264, 280, 305, 308, 312, 320, 324, 354, 365, 369, 370, 399, 421, 431, 442, 465, 476, 484, 488, 505, 506, 508, 523, 538, 539, 542, 569, 590, 594, 598, 622, 624, 626, 639, 642, 671, 676, 685, 689, 690, 707, 709, 712, 719, 744, 787, 792, 798, 814, 817, 821, 830, 842, 877, 893, 911, 923, 971, 978, 983, 994, 995, 1026, 1033]
KICK_L
[57, 149, 185, 346]
MAUL
[82, 102, 116, 207, 224, 265, 560, 586, 650, 752]
SCRUM
[84, 121, 141, 155, 215, 216, 226, 334, 340, 389, 391, 470, 534, 536, 718, 796, 841, 959, 1013, 1054]
TACKLE_S
[3, 6, 14, 18, 25, 29, 35, 49, 62, 74, 92, 94, 101, 124, 127, 138, 139, 146, 151, 158, 165, 169, 173, 179, 183, 192, 200, 209, 231, 234, 237, 242, 246, 250, 254, 258, 274, 278, 287, 295, 302, 306, 310, 314, 318, 322, 330, 367, 373, 377, 387, 393, 397, 406, 433, 435, 448, 450, 452, 472, 486, 490, 498, 502, 503, 511, 522, 525, 532, 540, 544, 567,